# Lab 3: Amazon AgentCore Identity を使用したエージェントの作成 

## 概要

このLabでは、Amazon Bedrock AgentCore Identity 機能を統合して、既存のカスタマーサポートエージェントを強化します。これにより、エージェントは AgentCore の Identity プロバイダーを通じて適切な資格情報管理を維持しながら、OAuth2 フローを使用して Google カレンダーなどの外部サービスと安全に認証できるようになります。

カレンダー統合により、サポートエージェントは顧客との会話内で製品デモやテクニカルサポートの予約などのイベントを直接スケジュールでき、サポートワークフローを合理化し、全体的な顧客体験を向上させます。

![エージェントアーキテクチャ](images/architecture_lab6_identity.png)

**ベース**: [公式カスタマーサポートアシスタント](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/02-use-cases/customer-support-assistant)


### Lab の詳細

| 情報        | 詳細                                                                          |
| :----------------- | :------------------------------------------------------------------------------- |
| Lab タイプ           | 増分強化                                                                   |
| エージェントタイプ         | 単一                                                                           |
| エージェントフレームワーク  | Strands Agents                                                                   |
| LLM モデル          | Anthropic Claude Haiku 4.5                                                      |
| Lab コンポーネント     | AgentCore Identity、OAuth2 プロバイダー、Google Calendar API、Cognito 統合  |
| Lab の業種       | カスタマーサポート                                                                 |
| 複雑さ | 中程度                                                                     |
| 使用 SDK           | Amazon BedrockAgentCore Python SDK、Strands Agents、Google API Client          |

### Lab のアーキテクチャ

このLabでは、Identity 管理機能を使用してカスタマーサポートエージェントを拡張します。エージェントは AgentCore Identity プロバイダーを通じてユーザーを認証し、認証されたユーザーに代わって Google カレンダーなどの外部サービスにアクセスします。

エージェントは以下を統合します:
- **AgentCore Identity**: 安全な資格情報管理と OAuth2 フロー
- **Google OAuth2 プロバイダー**: Google サービスとの認証
- **カレンダーツール**: イベントの作成とカレンダー情報の取得
- **Cognito プロバイダー**（オプション）: カスタム Identity プロバイダー統合


### Lab の主な機能

- 安全な OAuth2 認証フロー
- 適切な資格情報管理による外部サービス統合
- Google Calendar API 統合
- カスタム Identity プロバイダー設定

## 前提条件

このチュートリアルを実行するには、以下が必要です:

- Python 3.10+
- AWS 認証情報が設定済み
- Amazon Bedrock AgentCore SDK
- Strands Agents
- **Lab 1 & 2 を事前に完了** - このLabは既存のエージェント上に直接構築されます
- **Google Developer Console へのアクセス** - OAuth2 認証情報の作成用
- **AgentCore Identity 権限** - AgentCore Identity アクセス権を持つ IAM ロール

**注意**: このLabに進む前に、Lab 1 & 2 の基本エージェントが正しく動作していることを確認してください。

## ステップ 1: 依存関係のインストールとライブラリのインポート

AgentCore Identity 統合、Google API アクセス、OAuth2 認証フローに必要なパッケージをインストールします。また、後で使用するためのヘルパー関数も作成します。

In [ ]:
# Install required packages
%pip install strands-agents strands-agents-tools "boto3>=1.39.15" python-dotenv utils google-auth google-api-python-client ddgs -q

In [ ]:
# Import libraries
import boto3
import click
import sys
import json
import os
from botocore.exceptions import ClientError

from bedrock_agentcore.identity.auth import requires_access_token
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from datetime import datetime, timedelta
from strands import tool
from strands import Agent
from strands.models import BedrockModel
import webbrowser

from lab_helpers.utils import get_ssm_parameter

session = boto3.session.Session()
region = session.region_name

identity_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=region,
)
cognito = boto3.client("cognito-idp")
ssm = boto3.client("ssm", region_name=region)

print("✅ Libraries imported successfully!")

In [ ]:
# Helper functions to save, retrieve, and delete provider names from SSM


def store_provider_name_in_ssm(provider_name: str):
    """Store credential provider name in SSM parameter."""
    param_name = "/app/customersupport/agentcore/google_provider"
    try:
        ssm.put_parameter(
            Name=param_name, Value=provider_name, Type="String", Overwrite=True
        )
        click.echo(f"🔐 Stored provider name in SSM: {param_name}")
    except ClientError as e:
        click.echo(f"⚠️ Failed to store provider name in SSM: {e}")


def get_provider_name_from_ssm() -> str:
    """Get credential provider name from SSM parameter."""
    param_name = "/app/customersupport/agentcore/google_provider"
    try:
        response = ssm.get_parameter(Name=param_name)
        return response["Parameter"]["Value"]
    except ClientError:
        return None


def delete_ssm_param():
    """Delete SSM parameter for provider."""
    param_name = "/app/customersupport/agentcore/google_provider"
    try:
        ssm.delete_parameter(Name=param_name)
        click.echo(f"🧹 Deleted SSM parameter: {param_name}")
    except ClientError as e:
        click.echo(f"⚠️ Failed to delete SSM parameter: {e}")

## ステップ 2: カスタマーサポートツールのインポート

エージェントのコア機能を維持するために、Lab 1 のカスタマーサポートツールを再利用します。

## ステップ 3: AgentCore Identity クライアントの設定

このステップでは、Google Calendar API 機能をカスタマーサポートエージェントに統合します。ユーザーに代わって Google カレンダーなどの外部サービスに安全にアクセスするには、適切な認証メカニズムを実装する必要があります。

Amazon Bedrock AgentCore Identity は、手動のトークン管理、リフレッシュ処理、資格情報の保存の複雑さを排除し、OAuth2 認証フローを管理するための合理化されたアプローチを提供します。このサービスは、エージェントと外部サービスプロバイダー間の安全な仲介者として機能します。

Google OAuth クライアントを作成するには、以下の手順に従ってください:

#### 1. Google Developer Console でプロジェクトを作成

1. [Google Developer Console](https://console.developers.google.com/) に移動します。
2. 上部のナビゲーションバーで「Create Project」をクリックします。
3. プロジェクト名を入力します。
4. 組織を選択するか、該当しない場合は「No organization」のままにします。
5. 「Create」をクリックします。

新しいプロジェクトがプロジェクトリストに表示されます。

#### 2. Google Calendar API を有効化

1. プロジェクトを選択した状態で、左側のメニューを開き、APIs & Services > Library に移動します。
2. 検索バーに「Google Calendar API」と入力します。
3. 結果から Google Calendar API をクリックします。
4. 「Enable」をクリックします。

#### 3. OAuth 同意画面の設定

1. 左側のメニューで APIs & Services > OAuth consent screen に移動します。

2. 「Get started」をクリックします。

3. 必須フィールドを入力します: App Name、User Support Email
4. 「Next」をクリックし、ユーザータイプ（Internal または External）を選択します。External を選択する場合は、テスターのメールアドレスを追加します。開発者連絡先情報（メールアドレス）を提供します。
5. 規約に同意し、「Finish」をクリックします。
6. 「Create」をクリックして同意画面を確定します。

#### 4. OAuth 2.0 認証情報の作成

1. 左側のメニューから APIs & Services > Credentials に移動します。
2. Create Credentials > OAuth client ID をクリックします。
3. アプリケーションタイプとして「Web application」を選択します。
4. 認証情報の名前を入力します。
5. Authorized redirect URIs に以下のリダイレクト URI を追加します:
   - `https://bedrock-agentcore.us-east-1.amazonaws.com/identities/oauth2/callback`
6. 「Create」をクリックします。

#### 5. クライアント ID とクライアントシークレットの取得

作成後、ダイアログにクライアント ID とクライアントシークレットが表示されます。「Download JSON」をクリックして認証情報をファイルに保存します。このファイルをプロジェクトに `credentials.json` として保存します。（ファイル名を一致させるためにリネームが必要な場合があります）

#### 6. データアクセススコープの更新

1. APIs & Services > Credentials に移動します。
2. 作成した OAuth 2.0 client ID をクリックします。
3. 左側のメニューで「Data access」を選択します。
4. 「Add or remove scopes」をクリックします。
5. 「Manually add scopes」でスコープを入力します: `https://www.googleapis.com/auth/calendar`
6. 「Update」をクリックし、「Save」をクリックして設定を確認します。

In [ ]:
credentials_file = "credentials.json"

# Verify credentials file looks as expected, and extract credentials

if not os.path.isfile(credentials_file):
    print(f"❌ Error: '{credentials_file}' file not found")
    sys.exit(1)

print(f"📄 Reading credentials from {credentials_file}...")
try:
    with open(credentials_file, "r") as f:
        data = json.load(f)
except json.JSONDecodeError as e:
    print(f"❌ Error parsing JSON: {e}")
    sys.exit(1)

web_config = data.get("web")
if not web_config:
    print("❌ Error: 'web' section missing in credentials.json")
    sys.exit(1)

client_id = web_config.get("client_id")
client_secret = web_config.get("client_secret")

if not client_id:
    print("❌ Error: 'client_id' not found in credentials.json")
    sys.exit(1)

if not client_secret:
    print("❌ Error: 'client_secret' not found in credentials.json")
    sys.exit(1)

print("✅ Client ID and Secret loaded from credentials.json")

In [ ]:
google_provider_name = "customersupport-google-calendar"

try:
    print("🔧 Creating Google OAuth2 credential provider...")
    google_provider = identity_client.create_oauth2_credential_provider(
        name=google_provider_name,
        credentialProviderVendor="GoogleOauth2",
        oauth2ProviderConfigInput={
            "googleOauth2ProviderConfig": {
                "clientId": client_id,
                "clientSecret": client_secret,
            }
        },
    )

    print("✅ Google OAuth2 credential provider created successfully")
    google_provider_arn = google_provider["credentialProviderArn"]
    print(f"   Provider ARN: {google_provider_arn}")
    print(f"   Provider Name: {google_provider['name']}")

    # Store provider name in SSM
    store_provider_name_in_ssm(google_provider_name)
except Exception as e:
    print(f"❌ Error creating Google credential provider: {str(e)}")

In [ ]:
# List all OAuth2 credential providers.
try:
    response = identity_client.list_oauth2_credential_providers(maxResults=20)
    providers = response.get("credentialProviders", [])
    print(providers)
except Exception as e:
    print(f"❌ Error listing credential providers: {str(e)}", err=True)

次に、AgentCore Identity を使用して Google カレンダーで認証し、ユーザーに代わってカレンダー操作を実行するツールを作成します。

まず、OAuth2 認証フローを設定しましょう:

In [ ]:
async def on_auth_url(url: str):
    webbrowser.open(url)


SCOPES = ["https://www.googleapis.com/auth/calendar"]

google_access_token = None


@requires_access_token(
    provider_name=google_provider_name,
    scopes=["https://www.googleapis.com/auth/calendar"],  # Google OAuth2 scopes
    auth_flow="USER_FEDERATION",  # On-behalf-of user (3LO) flow
    on_auth_url=on_auth_url,  # prints authorization URL to console
    force_authentication=True,
    into="access_token",
)
def get_google_access_token(access_token: str):
    return access_token

次に、カレンダー管理ツールを作成しましょう:

In [ ]:
@tool(
    name="Create_calendar_event",
    description="Creates a new event on your Google Calendar",
)
def create_calendar_event() -> str:
    google_access_token = ""
    try:
        google_access_token = get_google_access_token(access_token=google_access_token)
        if not google_access_token:
            raise Exception("requires_access_token did not provide tokens")
    except Exception as e:
        return "Error Authentication with Google: " + str(e)

    creds = Credentials(token=google_access_token, scopes=SCOPES)

    try:
        service = build("calendar", "v3", credentials=creds)

        # Define event details
        start_time = datetime.now() + timedelta(hours=1)
        end_time = start_time + timedelta(hours=1)

        event = {
            "summary": "Test Event from API",
            "location": "Virtual",
            "description": "This event was created using the Google Calendar API.",
            "start": {
                "dateTime": start_time.isoformat() + "Z",  # UTC time
                "timeZone": "UTC",
            },
            "end": {
                "dateTime": end_time.isoformat() + "Z",
                "timeZone": "UTC",
            },
        }

        created_event = (
            service.events().insert(calendarId="primary", body=event).execute()
        )

        return json.dumps(
            {
                "event_created": True,
                "event_id": created_event.get("id"),
                "htmlLink": created_event.get("htmlLink"),
            }
        )

    except HttpError as error:
        return json.dumps({"error": str(error), "event_created": False})
    except Exception as e:
        return json.dumps({"error": str(e), "event_created": False})

In [ ]:
@tool(
    name="Get_calendar_events_today",
    description="Retrieves the calendar events for the day from your Google Calendar",
)
def get_calendar_events_today() -> str:
    google_access_token = ""
    try:
        google_access_token = get_google_access_token(access_token=google_access_token)

        if not google_access_token:
            raise Exception("requires_access_token did not provide tokens")
    except Exception as e:
        return "Error Authentication with Google: " + str(e)

    # Create credentials from the provided access token
    creds = Credentials(token=google_access_token, scopes=SCOPES)
    try:
        service = build("calendar", "v3", credentials=creds)
        # Call the Calendar API
        today_start = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        today_end = today_start.replace(hour=23, minute=59, second=59)

        # Format with CDT timezone (-05:00)
        timeMin = today_start.strftime("%Y-%m-%dT00:00:00-05:00")
        timeMax = today_end.strftime("%Y-%m-%dT23:59:59-05:00")

        events_result = (
            service.events()
            .list(
                calendarId="primary",
                timeMin=timeMin,
                timeMax=timeMax,
                singleEvents=True,
                orderBy="startTime",
            )
            .execute()
        )
        events = events_result.get("items", [])
        if not events:
            return json.dumps({"events": []})  # Return empty events array as JSON

        return json.dumps({"events": events})  # Return events wrapped in an object
    except HttpError as error:
        error_message = str(error)
        return json.dumps({"error": error_message, "events": []})
    except Exception as e:
        error_message = str(e)
        return json.dumps({"error": error_message, "events": []})

新しいカレンダー機能を備えたカスタマーサポートエージェントを作成しましょう:

In [ ]:
model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
system_prompt = """
    You are a helpful and professional customer support assistant for an electronics e-commerce company.
Your role is to:
- Provide accurate information using the tools available to you
- Support the customer with technical information and product specifications.
- Be friendly, patient, and understanding with customers
- Always offer additional help after answering questions
- If you can't help with something, direct customers to the appropriate contact

You have access to the following tools:
1. get_return_policy() - For warranty and return policy questions
2. get_product_info() - To get information about a specific product
3. web_search() - To access current technical documentation, or for updated information. 
4. create_calendar_event() - To create a new calendar event
5. get_calendar_events_today() - To find events on the calendar today
Always use the appropriate tool to get accurate, up-to-date information rather than making assumptions about electronic products or specifications.
    """
agent = Agent(
    model=model,
    system_prompt=system_prompt,
    tools=[create_calendar_event, get_calendar_events_today],
    callback_handler=None,
)

エージェントをテストする時が来ました！```Polling for token for authorization url``` というメッセージが表示され、その後に URL が続きます。この URL をクリックして Google アカウントにサインインし、エージェントに Google カレンダーへのアクセス権限を付与してください。

In [ ]:
print(
    str(
        agent(
            "Can you create a new event on my cal? You can call the create_calendar_event directly."
        )
    )
)

In [ ]:
print(str(agent("Whats my agenda for today?")))

## おめでとうございます！

**Amazon AgentCore Identity 機能を使用したエージェントの作成**を正常に完了しました！

### 達成したこと:

- **AgentCore Identity の設定**: 安全な認証のための OAuth2 資格情報プロバイダーをセットアップ  
- **Google カレンダー統合**: カレンダーイベント管理と表示のためのツールを作成  
- **カスタマーサポートの強化**: スケジューリングとカレンダー機能でエージェントを拡張  

### 主な学習内容:
- **Identity 管理**: 安全な資格情報管理のための AgentCore Identity の使用
- **OAuth2 フロー**: ユーザーフェデレーションとトークンベースの認証の実装
- **外部 API 統合**: Google カレンダーなどのサードパーティサービスへの安全な接続
- **ツール強化**: 既存のエージェントツールへの Identity 対応機能の追加

## 次のステップ

エージェントを強化する準備はできましたか？以下を続けてください:

- **Lab 4**: Gateway を活用してツールやその他のリソースを安全に接続
- **Lab 5**: 本番監視のためのオブザーバビリティとガードレールを実装
- **Lab 6**: スケーラブルな本番ホスティングのために AgentCore Runtime にデプロイ

## リソース

- [AgentCore Identity ドキュメント](https://docs.aws.amazon.com/bedrock/latest/userguide/agentcore-identity.html)
- [Google Calendar API ドキュメント](https://developers.google.com/calendar/api)
- [Strands Agents ドキュメント](https://github.com/strands-agents/sdk-python)
- [Amazon Bedrock モデル](https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html)
- [公式カスタマーサポートサンプル](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/02-use-cases/customer-support-assistant)

---

**素晴らしい仕事です！カスタマーサポートエージェントに安全な Identity 管理とカレンダー統合機能が追加されました！**

## （オプション）: Cognito で Identity プロバイダーを作成

追加の Identity 管理機能として、Cognito ベースの Identity プロバイダーも設定できます。このセクションでは、Amazon Cognito を使用してカスタム OAuth2 プロバイダーを作成する方法を示します:

In [ ]:
cognito_provider_name = "customersupport-gateways-cognito"

try:
    print("📥 Fetching Cognito configuration from SSM...")

    client_id = get_ssm_parameter("/app/customersupport/agentcore/client_id")
    print(f"✅ Retrieved client ID: {client_id}")

    client_secret = get_ssm_parameter("/app/customersupport/agentcore/cognito_secret")
    print(f"✅ Retrieved client secret: {client_secret[:4]}***")

    issuer = get_ssm_parameter("/app/customersupport/agentcore/cognito_discovery_url")
    auth_url = get_ssm_parameter("/app/customersupport/agentcore/cognito_auth_url")
    token_url = get_ssm_parameter("/app/customersupport/agentcore/cognito_token_url")

    print(f"✅ Issuer: {issuer}")
    print(f"✅ Authorization Endpoint: {auth_url}")
    print(f"✅ Token Endpoint: {token_url}")

    print("⚙️  Creating OAuth2 credential provider...")

    cognito_provider = identity_client.create_oauth2_credential_provider(
        name=cognito_provider_name,
        credentialProviderVendor="CustomOauth2",
        oauth2ProviderConfigInput={
            "customOauth2ProviderConfig": {
                "clientId": client_id,
                "clientSecret": client_secret,
                "oauthDiscovery": {
                    "authorizationServerMetadata": {
                        "issuer": issuer,
                        "authorizationEndpoint": auth_url,
                        "tokenEndpoint": token_url,
                        "responseTypes": ["code", "token"],
                    }
                },
            }
        },
    )

    provider_arn = cognito_provider["credentialProviderArn"]
    print(provider_arn)
except Exception as e:
    print(f"❌ Error creating Cognito credential provider: {str(e)}")

In [ ]:
response = identity_client.list_oauth2_credential_providers(maxResults=20)
providers = response.get("credentialProviders", [])
print(providers)

## クリーンアップ

In [ ]:
#  try:
#     print(f"🗑️  Deleting Google OAuth2 credential provider: {google_provider_name}")
#     identity_client.delete_oauth2_credential_provider(name=google_provider_name)
#     print("✅ Google OAuth2 credential provider deleted successfully")
# except Exception as e:
#     print(f"❌ Error deleting credential provider: {str(e)}")

In [ ]:
#  try:
#     print(f"🗑️  Deleting Cognito OAuth2 credential provider: {cognito_provider_name}")
#     identity_client.delete_oauth2_credential_provider(name=cognito_provider_name)
#     print("✅ Cognito credential provider deleted successfully")
# except Exception as e:
#     print(f"❌ Error deleting credential provider: {str(e)}")